In [ ]:
! pip install transformers

     |████████████████████████████████| 2.8 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 30.4 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 3.3 MB 40.1 MB/s 
     |████████████████████████████████| 636 kB 69.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
! pip install torch-scatter

  Created wheel for torch-scatter: filename=torch_scatter-2.0.8-cp37-cp37m-linux_x86_64.whl size=281098 sha256=6d1d3a0df39dae8e98a772cd22e9b143bf84b357d6c0477c74556f346f134216
  Stored in directory: /root/.cache/pip/wheels/96/e4/4e/2bcc6de6a801960aedbca43f7106d268f766c3f9f8ab49b3a5
Successfully built torch-scatter


In [ ]:
from transformers import TapasTokenizer, TapasForQuestionAnswering, TapasConfig
import pandas as pd
import torch

# model_name = 'google/tapas-base'
model_name = 'google/tapas-base-finetuned-wtq'

configuration = TapasConfig(num_aggregation_labels=5)
model = TapasForQuestionAnswering.from_pretrained(model_name)
tokenizer = TapasTokenizer.from_pretrained(model_name)

In [ ]:
df = pd.read_csv("test.csv")

In [ ]:
df.drop(axis = 1 , labels = ['Unknown' , 'xlabel' , 'ylabel' ,'title' , 'legend orientation'] , inplace= True)

In [ ]:
df

,Sex,Algen'a,Morocco,Cote d'Ivoire
0,Female,94.328795,66.396395,52.371431
1,Male,95.327128,73.505584,58.671080


In [ ]:
df = df.astype(str)

In [ ]:
data = df.to_dict(orient='list')

In [ ]:
queries = [ "What is the average survival rate for Morocco",
           "sum of survival rate for Morocco", 
           "how many countries are present",
           "What is the median survival rate for Morocco"]

In [ ]:
table = pd.DataFrame.from_dict(data)
inputs = tokenizer(table=table, queries=queries, padding='max_length', return_tensors="pt")
outputs = model(**inputs)
predicted_answer_coordinates, predicted_aggregation_indices = tokenizer.convert_logits_to_predictions(
        inputs,
        outputs.logits.detach(),
        outputs.logits_aggregation.detach()
)
print(predicted_answer_coordinates, predicted_aggregation_indices)
# let's print out the results:
id2aggregation = {0: "NONE", 1: "SUM", 2: "AVERAGE", 3:"COUNT", 4:"MEDIAN"}
aggregation_predictions_string = [id2aggregation[x] for x in predicted_aggregation_indices]

answers = []
for coordinates in predicted_answer_coordinates:
  if len(coordinates) == 1:
    # only a single cell:
    answers.append(table.iat[coordinates[0]])
  else:
    # multiple cells
    cell_values = []
    for coordinate in coordinates:
       cell_values.append(table.iat[coordinate])
    answers.append(", ".join(cell_values))

display(table)
print("")
for query, answer, predicted_agg in zip(queries, answers, aggregation_predictions_string):
  print(query)
  if predicted_agg == "NONE":
    print("Predicted answer: " + answer)
  else:
    print("Predicted answer: " + predicted_agg + " > " + answer)
  print("\n\n")

[[(0, 2), (1, 2)], [(0, 2), (1, 2)], [(0, 0), (1, 0)], [(1, 2)]] [2, 1, 3, 2]


,Sex,Algen'a,Morocco,Cote d'Ivoire
0,Female,94.32879505594684,66.39639456793715,52.371431164192636
1,Male,95.32712755875369,73.50558381424864,58.671079659884775



What is the average survival rate for Morocco
Predicted answer: AVERAGE > 66.39639456793715, 73.50558381424864



sum of survival rate for Morocco
Predicted answer: SUM > 66.39639456793715, 73.50558381424864



how many countries are present
Predicted answer: COUNT > Female, Male



What is the median survival rate for Morocco
Predicted answer: AVERAGE > 73.50558381424864



